


<h1>TCC 1 - Requisição e importação da série histórica de vazões</h1>

Esse notebook apresenta o código criado para importação de dados hidrometeorológicos do sistema Hidroweb, da Agência Nacional de Águas

# Requisição dos Dados

## Importação das bibliotecas

In [1]:
# Biblioteca de requisição HTTP
import requests
# Biblioteca para ler arquivos XML
import xml.etree.ElementTree as ET
import pandas as pd

## Definição e requisição dos dados

In [2]:
# Definição dos dados de entrada da requisição realizada no webservice da ANA (HIDRO)
codEstacao = 56075000
data_inicio = '01/01/1941'
data_fim = '31/12/2014'
tipo_dado = 3
nivel_consistencia = 2

In [3]:
# URL da requisição 
url_hidro = f"http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica?codEstacao={codEstacao}\
&dataInicio={data_inicio}&dataFim={data_fim}&tipoDados={tipo_dado}&nivelConsistencia={nivel_consistencia}"

In [4]:
url_hidro

'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica?codEstacao=56075000&dataInicio=01/01/1941&dataFim=31/12/2014&tipoDados=3&nivelConsistencia=2'

In [5]:
# Requisição HTML por meio da biblioteca requests
resp = requests.get(url_hidro)

In [6]:
# Resposta (200) indicando que a requisição foi bem sucedida
resp

<Response [200]>

In [7]:
# Corpo da resposta da API em bytes (XML)
resp.content

b'<?xml version="1.0" encoding="utf-8"?>\r\n<DataTable xmlns="http://MRCS/">\r\n  <xs:schema id="NewDataSet" xmlns="" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:msdata="urn:schemas-microsoft-com:xml-msdata">\r\n    <xs:element name="NewDataSet" msdata:IsDataSet="true" msdata:MainDataTable="SerieHistorica" msdata:UseCurrentLocale="true">\r\n      <xs:complexType>\r\n        <xs:choice minOccurs="0" maxOccurs="unbounded">\r\n          <xs:element name="SerieHistorica">\r\n            <xs:complexType>\r\n              <xs:sequence>\r\n                <xs:element name="EstacaoCodigo" type="xs:int" minOccurs="0" />\r\n                <xs:element name="NivelConsistencia" type="xs:unsignedByte" minOccurs="0" />\r\n                <xs:element name="DataHora" type="xs:string" minOccurs="0" />\r\n                <xs:element name="MediaDiaria" type="xs:string" minOccurs="0" />\r\n                <xs:element name="MetodoObtencaoVazoes" type="xs:string" minOccurs="0" />\r\n                <x

In [8]:
# Leitura da resposta pela biblioteca xml
root  = ET.XML(resp.content)

In [9]:
# Criado objeto do tipo Element pertencente à biblioteca xml
root

<Element '{http://MRCS/}DataTable' at 0x7f910ac5fbf0>

# Manipulação dos dados e criação do dataframe final

## Criação do dataframe

In [10]:
serie_historica = []

# Armazenamento dos dados do arquivo XML em uma lista 
for elem in root:
 
  for estacoes in elem:

    for dado in estacoes:
      dado_mensal = {}

      for prop in dado:
        dado_mensal[prop.tag] = prop.text
      
      serie_historica.append(dado_mensal)

In [11]:
serie_historica[:2]

[{'{http://www.w3.org/2001/XMLSchema}choice': '\n          '},
 {'DataHora': '2014-12-01 00:00:00',
  'DataIns': '2017-11-30 00:00:00',
  'DiaMaxima': '14',
  'DiaMinima': '13',
  'EstacaoCodigo': '56075000',
  'Maxima': '92.7253',
  'MaximaStatus': '1',
  'Media': '51.6681',
  'MediaAnual': '0',
  'MediaAnualStatus': '0',
  'MediaDiaria': '1',
  'MediaStatus': '1',
  'MetodoObtencaoVazoes': '1',
  'Minima': '25.3155',
  'MinimaStatus': '1',
  'NivelConsistencia': '2',
  'Vazao01': '87.5085',
  'Vazao01Status': '1',
  'Vazao02': '69.4007',
  'Vazao02Status': '1',
  'Vazao03': '52.8741',
  'Vazao03Status': '1',
  'Vazao04': '41.1192',
  'Vazao04Status': '1',
  'Vazao05': '44.9707',
  'Vazao05Status': '1',
  'Vazao06': '80.8928',
  'Vazao06Status': '1',
  'Vazao07': '87.5085',
  'Vazao07Status': '1',
  'Vazao08': '76.5418',
  'Vazao08Status': '1',
  'Vazao09': '41.1192',
  'Vazao09Status': '1',
  'Vazao10': '34.8609',
  'Vazao10Status': '1',
  'Vazao11': '31.2099',
  'Vazao11Status': '1'

In [12]:
# Retirando o 1º elemento da lista
serie_historica = serie_historica[1:]

In [13]:
# Armazenando os dados da lista em um DataFrame
df = pd.DataFrame(serie_historica)
df

,EstacaoCodigo,NivelConsistencia,DataHora,MediaDiaria,MetodoObtencaoVazoes,Maxima,Minima,Media,DiaMaxima,DiaMinima,...,Vazao23Status,Vazao24Status,Vazao25Status,Vazao26Status,Vazao27Status,Vazao28Status,Vazao29Status,Vazao30Status,Vazao31Status,DataIns
0,56075000,2,2014-12-01 00:00:00,1,1,92.7253,25.3155,51.6681,14,13,...,1,1,1,1,1,1,1,1,1,2017-11-30 00:00:00
1,56075000,2,2014-11-01 00:00:00,1,1,233.574,13.3478,42.7999,29,5,...,1,1,1,1,1,1,1,1,None,2017-11-30 00:00:00
2,56075000,2,2014-10-01 00:00:00,1,1,20.2371,10.3803,14.0579,27,15,...,1,1,1,1,1,1,1,1,1,2017-11-30 00:00:00
3,56075000,2,2014-09-01 00:00:00,1,1,20.7898,12.843,15.6866,5,26,...,1,1,1,1,1,1,1,1,None,2017-11-30 00:00:00
4,56075000,2,2014-08-01 00:00:00,1,1,23.0302,16.9861,19.0654,1,25,...,1,1,1,1,1,1,1,1,1,2017-11-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,56075000,2,1941-05-01 00:00:00,1,1,66.9951,57.0519,60.6999,1,29,...,1,1,1,1,1,1,1,1,1,2017-11-30 00:00:00
884,56075000,2,1941-04-01 00:00:00,1,1,161.108,66.9951,90.2891,4,30,...,1,1,1,1,1,1,1,1,None,2017-11-30 00:00:00
885,56075000,2,1941-03-01 00:00:00,1,1,156.794,78.6167,100.489,1,31,...,1,1,1,1,1,1,1,1,1,2017-11-30 00:00:00
886,56075000,2,1941-02-01 00:00:00,1,1,199.611,63.5967,91.6537,28,17,...,1,1,1,1,1,1,None,None,None,2017-11-30 00:00:00


## Manipulações iniciais no dataframe

In [14]:
# Definindo as colunas de interesse no DataFrame que serão utilizadas
colunas1 = ['Vazao0' + str(i) for i in range(1, 10)]
colunas2 = ['Vazao' + str(i) for i in range(10, 32)]
colunas = colunas1 + colunas2
colunas

['Vazao01',
 'Vazao02',
 'Vazao03',
 'Vazao04',
 'Vazao05',
 'Vazao06',
 'Vazao07',
 'Vazao08',
 'Vazao09',
 'Vazao10',
 'Vazao11',
 'Vazao12',
 'Vazao13',
 'Vazao14',
 'Vazao15',
 'Vazao16',
 'Vazao17',
 'Vazao18',
 'Vazao19',
 'Vazao20',
 'Vazao21',
 'Vazao22',
 'Vazao23',
 'Vazao24',
 'Vazao25',
 'Vazao26',
 'Vazao27',
 'Vazao28',
 'Vazao29',
 'Vazao30',
 'Vazao31']

In [15]:
# Criando outro DataFrame somente com as colunas de interesse
df2 = df[['DataHora'] + colunas]
df2

,DataHora,Vazao01,Vazao02,Vazao03,Vazao04,Vazao05,Vazao06,Vazao07,Vazao08,Vazao09,...,Vazao22,Vazao23,Vazao24,Vazao25,Vazao26,Vazao27,Vazao28,Vazao29,Vazao30,Vazao31
0,2014-12-01 00:00:00,87.5085,69.4007,52.8741,41.1192,44.9707,80.8928,87.5085,76.5418,41.1192,...,32.4178,39.2199,42.3953,77.9867,58.9647,44.9707,39.2199,25.3155,28.8224,31.2099
1,2014-11-01 00:00:00,16.4557,15.4053,14.3691,14.3691,13.3478,13.3478,14.3691,14.3691,15.4053,...,14.8854,16.4557,17.5199,55.5645,58.9647,62.405,133.236,233.574,218.322,None
2,2014-10-01 00:00:00,12.843,14.3691,14.8854,14.3691,13.3478,12.843,12.843,12.3421,12.3421,...,17.5199,16.9861,16.4557,17.5199,17.5199,20.2371,19.6874,19.6874,19.1407,16.9861
3,2014-09-01 00:00:00,19.6874,19.6874,19.6874,19.6874,20.7898,19.6874,18.0569,17.5199,16.4557,...,14.3691,14.3691,13.8566,13.3478,12.843,12.843,12.843,12.843,12.843,None
4,2014-08-01 00:00:00,23.0302,23.0302,21.9042,20.7898,19.6874,19.6874,19.6874,19.1407,18.5972,...,19.1407,18.5972,17.5199,16.9861,16.9861,17.5199,17.5199,17.5199,18.0569,18.0569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,1941-05-01 00:00:00,66.9951,65.2854,64.4385,64.4385,63.5967,62.7602,61.929,61.103,61.103,...,59.4667,59.4667,59.4667,58.6565,58.6565,57.8515,57.8515,57.0519,57.8515,61.103
884,1941-04-01 00:00:00,77.6916,99.5782,143.093,161.108,144.129,119.114,98.6825,89.1353,89.1353,...,74.9472,73.1436,72.2496,70.4772,69.5989,69.5989,68.7257,67.8578,66.9951,None
885,1941-03-01 00:00:00,156.794,130.887,115.287,108.709,117.194,114.338,100.477,95.1317,87.1764,...,86.2046,82.3693,79.5471,86.2046,98.6825,121.046,117.194,95.1317,81.4234,78.6167
886,1941-02-01 00:00:00,91.1149,91.1149,83.3204,79.5471,78.6167,76.7716,72.2496,70.4772,68.7257,...,114.338,109.64,105.019,114.338,132.891,190.261,199.611,None,None,None


## Manipulações relacionadas às datas

In [16]:
# Criando um Dataframe com mudança de orientação do formato
df_melt = df2.melt(id_vars=['DataHora'])
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27528 entries, 0 to 27527
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DataHora  27528 non-null  object
 1   variable  27528 non-null  object
 2   value     26976 non-null  object
dtypes: object(3)
memory usage: 645.3+ KB


In [17]:
df_melt.tail()

,DataHora,variable,value
27523,1941-05-01 00:00:00,Vazao31,61.103
27524,1941-04-01 00:00:00,Vazao31,None
27525,1941-03-01 00:00:00,Vazao31,78.6167
27526,1941-02-01 00:00:00,Vazao31,None
27527,1941-01-01 00:00:00,Vazao31,94.1229


In [18]:
# Criando coluna 'Data' 
df_melt['Data'] = pd.to_datetime(df_melt['DataHora'])

In [19]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27528 entries, 0 to 27527
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DataHora  27528 non-null  object        
 1   variable  27528 non-null  object        
 2   value     26976 non-null  object        
 3   Data      27528 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 860.4+ KB


In [20]:
df_melt.head()

,DataHora,variable,value,Data
0,2014-12-01 00:00:00,Vazao01,87.5085,2014-12-01
1,2014-11-01 00:00:00,Vazao01,16.4557,2014-11-01
2,2014-10-01 00:00:00,Vazao01,12.843,2014-10-01
3,2014-09-01 00:00:00,Vazao01,19.6874,2014-09-01
4,2014-08-01 00:00:00,Vazao01,23.0302,2014-08-01


In [21]:
# from datetime import timedelta
import numpy as np 

In [22]:
# Definindo função para criar coluna com formato adequado de data (AAAA-MM-DD)
def analise_datas(row):
  data = row['Data']
  var = row['variable']
  if var == 'Vazao31' and data.month in (4, 6, 9, 11):
    result = None
  elif (not data.is_leap_year and data.month == 2 and var in ('Vazao29', 'Vazao30','Vazao31')) or\
    (data.is_leap_year and data.month == 2 and var in ('Vazao30','Vazao31')):
    result = None
  else:
    result = data + np.timedelta64(int(var[5:])-1, 'D')
  return result

In [23]:
# Aplicação da função 'analise_datas' no DataFrame
df_melt['Data2'] = df_melt.apply(analise_datas, axis=1)

In [24]:
df_melt.tail()

,DataHora,variable,value,Data,Data2
27523,1941-05-01 00:00:00,Vazao31,61.103,1941-05-01,1941-05-31
27524,1941-04-01 00:00:00,Vazao31,None,1941-04-01,NaT
27525,1941-03-01 00:00:00,Vazao31,78.6167,1941-03-01,1941-03-31
27526,1941-02-01 00:00:00,Vazao31,None,1941-02-01,NaT
27527,1941-01-01 00:00:00,Vazao31,94.1229,1941-01-01,1941-01-31


In [25]:
# Ordenando DataFrame por pela data criada na coluna 'Data2'
df_melt.sort_values('Data2', inplace=True)

In [26]:
df_melt.iloc[56:64]

,DataHora,variable,value,Data,Data2
23086,1941-02-01 00:00:00,Vazao26,132.891,1941-02-01,1941-02-26
23974,1941-02-01 00:00:00,Vazao27,190.261,1941-02-01,1941-02-27
24862,1941-02-01 00:00:00,Vazao28,199.611,1941-02-01,1941-02-28
885,1941-03-01 00:00:00,Vazao01,156.794,1941-03-01,1941-03-01
1773,1941-03-01 00:00:00,Vazao02,130.887,1941-03-01,1941-03-02
2661,1941-03-01 00:00:00,Vazao03,115.287,1941-03-01,1941-03-03
3549,1941-03-01 00:00:00,Vazao04,108.709,1941-03-01,1941-03-04
4437,1941-03-01 00:00:00,Vazao05,117.194,1941-03-01,1941-03-05


## Manipulações finais no Dataframe

In [27]:
# Criação do DataFrame final com apenas as colunas de data e vazão observada
df_final = df_melt[['Data2', 'value']].copy()

In [28]:
# Renomeando as colunas do DataFrame
df_final.columns = ['Data', 'Vazao']

In [29]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27528 entries, 887 to 27526
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Data    27028 non-null  datetime64[ns]
 1   Vazao   26976 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 645.2+ KB


In [30]:
df_final.tail()

,Data,Vazao
27517,NaT,None
27519,NaT,None
27522,NaT,None
27524,NaT,None
27526,NaT,None


In [31]:
# Deletando as linhas do DataFrame que não possuem data (e portanto vazão).
df_final.dropna(subset=['Data'], inplace=True)

In [32]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27028 entries, 887 to 26640
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Data    27028 non-null  datetime64[ns]
 1   Vazao   26976 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 633.5+ KB


In [33]:
df_final.tail()

,Data,Vazao
23088,2014-12-27,44.9707
23976,2014-12-28,39.2199
24864,2014-12-29,25.3155
25752,2014-12-30,28.8224
26640,2014-12-31,31.2099


In [34]:
# Redefinindo o tipo de dado da coluna 'Vazão' para float
df_final = df_final.astype({'Vazao': float})

In [35]:
# Resetando o índice do DataFrame para que fique coerente
# com a ordenação das datas
df_final.reset_index(drop=True)

,Data,Vazao
0,1941-01-01,103.1930
1,1941-01-02,106.8580
2,1941-01-03,127.9030
3,1941-01-04,178.8230
4,1941-01-05,145.1680
...,...,...
27023,2014-12-27,44.9707
27024,2014-12-28,39.2199
27025,2014-12-29,25.3155
27026,2014-12-30,28.8224


In [36]:
# Transformando o campo 'Data' em índice do DataFrame
df_final.set_index("Data",inplace=True)
df_final

,Vazao
Data,
1941-01-01,103.1930
1941-01-02,106.8580
1941-01-03,127.9030
1941-01-04,178.8230
1941-01-05,145.1680
...,...
2014-12-27,44.9707
2014-12-28,39.2199
2014-12-29,25.3155


In [37]:
# Exportando os dados do DataFrame para um arquivo CSV
df_final.to_csv('vazoes_diarias.csv')